# How to Talk to a PDF using LangChain and ChatGPT

In [ ]:
# !pip install langchain
# !pip install pypdf
# !pip install chromadb

In [24]:
!curl -o paper.pdf https://arxiv.org/pdf/2303.13519.pdf

/bin/bash: /home/lucassoares/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6745k  100 6745k    0     0   917k      0  0:00:07  0:00:07 --:--:-- 1107k


In [25]:
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import ChatVectorDBChain
from langchain.llms import OpenAI

In [26]:
pdf_path = "./paper.pdf"
loader = PyPDFLoader(pdf_path)
pages = loader.load_and_split()
print(pages[0].page_content)

Learning and Veriﬁcation of Task Structure in Instructional Videos
Medhini Narasimhan1;2, Licheng Yu2, Sean Bell2, Ning Zhang2, Trevor Darrell1
1UC Berkeley,2Meta AI
https://medhini.github.io/task_structure
Abstract
Given the enormous number of instructional videos
available online, learning a diverse array of multi-step task
models from videos is an appealing goal. We introduce
a new pre-trained video model, VideoTaskformer, focused
on representing the semantics and structure of instructional
videos. We pre-train VideoTaskformer using a simple and
effective objective: predicting weakly supervised textual la-
bels for steps that are randomly masked out from an instruc-
tional video (masked step modeling). Compared to prior
work which learns step representations locally, our ap-
proach involves learning them globally, leveraging video of
the entire surrounding task as context. From these learned
representations, we can verify if an unseen video correctly
executes a given task, as well a

In [27]:
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(pages, embedding=embeddings,persist_directory=".")
vectordb.persist()

Running Chroma using direct local API.
loaded in 22 embeddings
loaded in 1 collections
collection with name langchain already exists, returning existing collection
Persisting DB to disk, putting it in the save folder .


In [28]:
pdf_qa = ChatVectorDBChain.from_llm(OpenAI(temperature=0.9, model_name="gpt-3.5-turbo"),
                                    vectordb, return_source_documents=True)

query = "What is the VideoTaskformer?"
result = pdf_qa({"question": query, "chat_history": ""})
print("Answer:")
print(result["answer"])

PersistentDuckDB del, about to run persist
Persisting DB to disk, putting it in the save folder .
Answer:
The VideoTaskformer is a method for learning step representations through masking in order to predict the correct steps for a given task based on a sequence of video clips.
